In [7]:
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split() #➊
    except AttributeError: # 不能调用.replace或.split方法
        pass # 假定field_names本就是标识符组成的序列
    field_names = tuple(field_names) #➋
 
    def __init__(self, *args, **kwargs): #➌
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)
 
    def __iter__(self): #➍
        for name in self.__slots__:
            yield getattr(self, name)
 
    def __repr__(self): #➎
        values = ', '.join('{}={!r}'.format(*i) for i 
                           in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)
    
    cls_attrs = dict(__slots__ = field_names, #➏
                     __init__ = __init__,
                     __iter__ = __iter__,
                     __repr__ = __repr__)
 
    return type(cls_name, (object,), cls_attrs)

* ❶ 这里体现了鸭子类型：尝试在逗号或空格处拆分 field_names；如果失败，那么假定field_names 本就是可迭代的对象，一个元素对应一个属性名。
* ❷ 使用属性名构建元组，这将成为新建类的 \__slots\__ 属性；此外，这么做还设定了拆包和字符串表示形式中各字段的顺序。
* ❸ 这个函数将成为新建类的 \__init\__ 方法。参数有位置参数和（或）关键字参数。
* ❹ 实现 \__iter\__ 函数，把类的实例变成可迭代的对象；按照 \__slots\__ 设定的顺序产出字段值。
* ❺ 迭代 \__slots\__ 和 self，生成友好的字符串表示形式。
* ❻ 组建类属性字典。
* ❼ 调用 type 构造方法，构建新类，然后将其返回。

In [6]:
Dog = record_factory('Dog', 'name weight owner')
rex = Dog('Rex', 30, 'Bob')
rex

Dog(name='Rex', weight=30, owner='Bob')

In [11]:
Dog.__iter__?